In [ ]:
"""
Module Name: Hopping.ipynb
            

Author: Kuan H Hsu
Date: 2024-12-01
Description:
    - Calculates effective Raman scattering operator on a square lattice, projected onto different scattering geometry
    - This technique was introduced by Shastry and Shraiman: 
      S. Shastry and B. I. Shraiman, International Journal of Modern Physics B 05, 365 (1991).
"""


import numpy as np
import itertools

In [ ]:
def not_parallel(hop1,hop2):
    return abs(abs(np.dot(hop2,hop1)/np.linalg.norm(hop1)/np.linalg.norm(hop2))-1) > 1e-6

def all_parallel(path):
    hop1 = points_arr[path[1]]-points_arr[path[0]]
    for i in range(2,len(path)):
        hopn = points_arr[path[i]]-points_arr[path[i-1]]
        if (not_parallel(hop1,hopn)): return False
    return True

def check_hop(path):
    hn = [0,0,0,0]
    for i in range(len(path)-1):
        hop = np.linalg.norm(points_arr[path[i+1]]-points_arr[path[i]])
        if (abs(hop-1) < 1e-6): hn[0] += 1
#         elif (abs(hop-np.sqrt(2)) < 1e-6 or abs(hop-2) < 1e-6): hop_num[1] += 1
        elif (abs(hop-np.sqrt(2)) < 1e-6 or abs(hop-np.sqrt(3)) < 1e-6): hn[1] += 1 # Triangle
        elif (abs(hop-2) < 1e-6): hn[2] += 1 
        elif (abs(hop-np.sqrt(5)) < 1e-6): hn[3] += 1 
    return hn

def default_check(path): return True

def hash_dot(a,b,pts):
    # Hash a (S_a * S_b) term to a number
    if ((a > pts) or (b > pts)): return -1
    if (a == b): return -1
    return (a-1)*pts+(b-1)

def rhash_dot(num,pts):
    # Return a number to a (S_a * S_b) term
    if (num >= pts**2): return -1,-1
    a = int(num/pts)+1
    b = num - (a-1)*pts +1
    if (a == b): return -1,-1
    return a,b

def print_chiral(term_arr,pts,print_mode):
    pm = print_mode
    result = "A2g = "
    # Combine terms
    for n,t in enumerate(term_arr):
        if (t != 0):
            a,b,c = rhash_chiral(n,pts)
            pt_arr = np.asarray([a,b,c])
            max_ind = pt_arr.argmax()
            min_ind = pt_arr.argmin()
            if (max_ind == 0 and min_ind == 2): continue
            pt_arr = np.sort(pt_arr)
            newind = hash_chiral(pt_arr[2],pt_arr[1],pt_arr[0],pts)
            if ((max_ind-min_ind+3)%3 == 1):
                term_arr[newind] += t
            else:
                # Reverse order
                term_arr[newind] -= t
            term_arr[n] = 0
    count = 0
    tr_arr = [0,0,0,0,0,0,0,0,0,0,0]
#     print(term_arr)
    for n,t in enumerate(term_arr):
        if (abs(t) > 1e-4):
            a,b,c = rhash_chiral(n,pts)
            if (a == -1): continue
            # Kick out points in a straight line
            if (not not_parallel(points_arr[b-1]-points_arr[a-1],points_arr[c-1]-points_arr[a-1])): continue
            if (print_mode == 1):
                ss = "S_{" + str(a)+"," + str(b) +","+ str(c)+"}"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + fmt_cmplx(t,pm) + ss
                else: result = result + fmt_cmplx(t,pm) + ss
            elif (print_mode == 2):
                count = count + 1
                ss = "S_{r"+fmt_str(a-1)+",r"+fmt_str(b-1)+",r"+fmt_str(c-1)+"}"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + fmt_cmplx(t,pm) + ss
                else: result = result  + fmt_cmplx(t,pm) + ss
                if (count != 0 and (count-2) % 3 == 0): result = result + "\n \\\& "
            elif (print_mode == 3):
                count = count + 1
                ss = "rsdotks(q1,q2,q3,"+fmt_str(c-1,3)+","+fmt_str(b-1,3)+","+fmt_str(a-1,3)+")"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + fmt_cmplx(t,pm) + "*" + ss
                else: result = result + fmt_cmplx(t,pm) + "*" + ss
                if (count != 0 and count % 2 == 0): result = result + " & \n"
            elif (print_mode == 4):  
                # Triangle order:
                # A: 1, sqrt(2), 1
                # B1: 1, sqrt(2), sqrt(3), 2x,y
                # B2: 1, sqrt(2), sqrt(3), x,2y
                # C: sqrt(2), 2, sqrt(2)
                # D1: 1, sqrt(5), 2, x, 2x+y
                # D2: 1, sqrt(5), 2, y, 2y+x
                # E1,2: sqrt(2),sqrt(5),3
                # F1,2: sqrt(2),2,sqrt(10)
                # G: 2,2,sqrt(8)
                pad = 0
                prefac = 1
                side_a = points_arr[c-1]-points_arr[b-1]
                side_b = points_arr[b-1]-points_arr[a-1]
                side_c = points_arr[a-1]-points_arr[c-1]
                len_a = np.linalg.norm(side_a)
                len_b = np.linalg.norm(side_b)
                len_c = np.linalg.norm(side_c)
                mat = np.asarray([points_arr[c-1],points_arr[b-1],points_arr[a-1]])
                mat = np.concatenate((mat, np.asarray([[1,1,1]]).T),1)
                flux = np.linalg.det(mat)
                ss = "S_{r"+fmt_str(a-1)+",r"+fmt_str(b-1)+",r"+fmt_str(c-1)+"}"
                if (set([len_a,len_b,len_c]).issubset([1.0,np.sqrt(2)])): 
                    if (flux > 0): tr_arr[0] += t
                    else: tr_arr[0] -= t
#                     print(ss,len_a,len_b,len_c,"Type A",flux,t)
                if (set([1.0,np.sqrt(2),np.sqrt(5)]).issubset([len_a,len_b,len_c])): 
                    side_arr = np.asarray([len_a,len_b,len_c])
                    pad = determine_triangle([side_a,side_b,side_c],side_arr.argmax(),[-2,-1])
                    if (flux > 0): tr_arr[1+pad] += t*prefac
                    else: tr_arr[1+pad] -= t*prefac
#                     print(ss,len_a,len_b,len_c,"Type B",flux,t,pad)
                if (set([len_a,len_b,len_c]).issubset([2.0,np.sqrt(2)])): 
                    if (flux > 0): tr_arr[3] += t
                    else: tr_arr[3] -= t
#                     print(ss,len_a,len_b,len_c,"Type C",flux,t)
                if (set([1.0,2.0,np.sqrt(5)]).issubset([len_a,len_b,len_c])): 
                    side_arr = np.asarray([len_a,len_b,len_c])
                    pad = determine_triangle([side_a,side_b,side_c],side_arr.argmax(),[-1,2])
                    if (flux > 0): tr_arr[4+pad] += t*prefac
                    else: tr_arr[4+pad] -= t*prefac
                if (set([np.sqrt(2.0),np.sqrt(5.0),3]).issubset([len_a,len_b,len_c])): 
                    side_arr = np.asarray([len_a,len_b,len_c])
                    pad = determine_triangle([side_a,side_b,side_c],np.argpartition(side_arr,2)[1],[-2,1])
                    if (flux > 0): tr_arr[6+pad] += t*prefac
                    else: tr_arr[6+pad] -= t*prefac
                if (set([np.sqrt(2.0),2,np.sqrt(10.0)]).issubset([len_a,len_b,len_c])): 
                    side_arr = np.asarray([len_a,len_b,len_c])
                    pad = determine_triangle([side_a,side_b,side_c],side_arr.argmax(),[-3,-1])
                    if (flux > 0): tr_arr[8+pad] += t*prefac
                    else: tr_arr[8+pad] -= t*prefac
                if (set([len_a,len_b,len_c]).issubset([2.0,np.sqrt(8)])): 
                    if (flux > 0): tr_arr[10] += t
                    else: tr_arr[10] -= t
#                     print(ss,len_a,len_b,len_c,"Type D",flux,t,pad)
        if (print_mode == 4 and n == len(term_arr)-1):
            print(tr_arr)
            result += fmt_cmplx(tr_arr[0],pm)+"S_{r+y,r+x,r}"
            result += fmt_cmplx(tr_arr[1],pm)+"S_{r+2x+y,r+x,r}"
            result += fmt_cmplx(tr_arr[2],pm)+"S_{r+y,r+2y+x,r}"
            result += fmt_cmplx(tr_arr[3],pm)+"S_{r+x,r+2x+y,r+y}"
            result += fmt_cmplx(tr_arr[4],pm)+"S_{r+2y,r+x,r}"
            result += fmt_cmplx(tr_arr[5],pm)+"S_{r+y,r+2x,r}"
            result += fmt_cmplx(tr_arr[6],pm)+"S_{r+3x,r+2x+y,r}"
            result += fmt_cmplx(tr_arr[7],pm)+"S_{r+3y,r+2y+x,r}"
            result += fmt_cmplx(tr_arr[8],pm)+"S_{r+3x+y,r+2x,r}"
            result += fmt_cmplx(tr_arr[9],pm)+"S_{r+3y+x,r+2y,r}"
            result += fmt_cmplx(tr_arr[10],pm)+"S_{r+2y,r+2x,r}"
    if (result == "A2g = "): result = "A2g = 0"
    print(result)
    return

def determine_triangle(side_arr,targetind,check):
    check1 = check
    check2 = [check[1],check[0]]
    for i in range(4):
        for n,s in enumerate(side_arr):
            side_arr[n] = rot90(s)
        if ((side_arr[targetind] == check1).all()): return 0
        if ((side_arr[targetind] == check2).all()): return 1
    return 0

def rot90(arr):
    return np.asarray([-arr[1],arr[0]])


def print_dot(term_arr,pts,print_mode,symm,order=0):
    if(symm == 0): result = "A1g = "
    elif(symm == 1): result = "A2g = "
    elif(symm == 2): result = "B1g = "
    elif(symm == 3): result = "B2g = "
    # Combine terms
#     for n,t in enumerate(term_arr):
#         if (t != 0):
#             a,b = rhash_dot(n,pts)
#             pt_arr = np.sort(np.asarray([a,b]))
#             newind = hash_dot(pt_arr[1],pt_arr[0],pts)
#             if (newind != n):
#                 term_arr[newind] += t
#                 term_arr[n] = 0
    count = 0
#     print(term_arr)
    hop_mat = np.zeros((5,5))
    for n,t in enumerate(term_arr):
        if (t != 0):
            a,b = rhash_dot(n,pts)
            if (a == -1): continue 
            if (print_mode == 1):
                if (t == 1): result = result + "+S_{" + str(a)+"," + str(b) +"}"
                elif (t == -1): result = result + "-S_{" + str(a)+"," + str(b)+"}"
                elif(t > 1): result = result + "+" + str(int(t)) + "S_{" + str(a)+"," + str(b)+"}"
                else: result = result + str(int(t)) + "S_{" + str(a)+"," + str(b) +"}"
            elif (print_mode == 2):
                count = count + 1
                ss = "S_{r"+fmt_str(a-1)+",r"+fmt_str(b-1)+"}"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + str(int(t)) + ss
                else: result = result + str(int(t)) + ss
                if (count != 0 and (count-2) % 3 == 0): result = result + "\n \\\& "
            elif (print_mode == 3):
                count = count + 1
                ss = "r2dotk2("+str(int(order))+",q1,q2,"+fmt_str(b-1,3)+","+fmt_str(a-1,3)+")"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + str(int(t)) + "*" + ss
                else: result = result + str(int(t)) + "*" + ss
                if (count != 0 and count % 2 == 0): result = result + " & \n"
            elif (print_mode == 4):
                ### Create a 2x2 grid for hashing
                hop = points_arr[a-1] - points_arr[b-1]
                hop_mat[hop[0],hop[1]] += t
#                 print(t,points_arr[a-1],points_arr[b-1],hop_mat)
        if (print_mode == 4 and n == len(term_arr)-2):
            for x in np.arange(2,-3,-1):
                for y in np.arange(2,-3,-1):
                    if (hop_mat[x,y] == 0): continue
                    result += fmt_cmplx(hop_mat[x,y],print_mode)+"S_{r"+fmt_cmplx(x)+"x"+fmt_cmplx(y)+"y,r}"
    if(symm == 0): 
        if (result == "A1g = "): result = "A1g = 0"
    elif(symm == 1): 
        if (result == "A2g = "): result = "A2g = 0"
    elif(symm == 2): 
        if (result == "B1g = "): result = "B1g = 0"
    elif(symm == 3): 
        if (result == "B2g = "): result = "B2g = 0"         
    print(result)
    return

def print_4term(term_arr,pts,print_mode,symm,order=0):
    pm = print_mode
    if(symm == 0): result = "A1g = "
    elif(symm == 1): result = "A2g = "
    elif(symm == 2): result = "B1g = "
    elif(symm == 3): result = "B2g = "
    # Combine terms
    for n,t in enumerate(term_arr):
        if (t != 0):
            a,b,c,d = rhash_4term(n,pts)
            if (a == b or c == d): term_arr[n] = 0
            hop1 = points_arr[b-1]-points_arr[a-1]
            hop2 = points_arr[c-1]-points_arr[b-1]
            hop3 = points_arr[d-1]-points_arr[c-1]
            hop4 = points_arr[a-1]-points_arr[d-1]
#             print(hop1,hop2,hop3,hop4,np.dot(np.cross(hop1,hop2),np.cross(hop3,hop4)))
    count = 0
#     print(term_arr)
    for n,t in enumerate(term_arr):
        if (t != 0):
            a,b,c,d = rhash_4term(n,pts)
            if (a == -1): continue 
            if (print_mode == 1):
                ss = "S_{" + str(a)+"," + str(b) +","+ str(c)+","+ str(d)+"}"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + fmt_cmplx(t,pm) + ss
                else: result = result + fmt_cmplx(t,pm) + ss
            elif (print_mode == 2):
                count = count + 1
                ss = "S_{r"+fmt_str(a-1)+",r"+fmt_str(b-1)+",r"+fmt_str(c-1)+",r"+fmt_str(d-1)+"}"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + str(int(t)) + ss
                else: result = result + str(int(t)) + ss
                if (count != 0 and (count-2) % 3 == 0): result = result + "\n \\\& "
            elif (print_mode == 3):
                count = count + 1
                ss = "r2dotk2("+str(int(order))+",q1,q2,"+fmt_str(b-1,3)+","+fmt_str(a-1,3)+")"
                if (t == 1): result = result + "+" + ss
                elif (t == -1): result = result + "-" + ss
                elif(t > 1): result = result + "+" + str(int(t)) + "*" + ss
                else: result = result + str(int(t)) + "*" + ss
                if (count != 0 and count % 2 == 0): result = result + " & \n"
    if(symm == 0): 
        if (result == "A1g = "): result = "A1g = 0"
    elif(symm == 1): 
        if (result == "A2g = "): result = "A2g = 0"
    elif(symm == 2): 
        if (result == "B1g = "): result = "B1g = 0"
    elif(symm == 3): 
        if (result == "B2g = "): result = "B2g = 0"         
    print(result)
    return

# def print_dot(term_arr,pts,print_mode,symm):
def pol(hopi,hopf,symm):
    C1 = 0
    C2 = 0
    if (symm == 0): # A1g
        C1 = np.dot([1,0],hopi)*np.dot([1,0],hopf)
        C2 = np.dot([0,1],hopi)*np.dot([0,1],hopf)
    elif (symm == 1): # A2g
        C1 = np.dot([1.0,0],hopi)*np.dot([0,1.0],hopf)
        C2 = -np.dot([0,1.0],hopi)*np.dot([1.0,0],hopf)
#         C1 = np.dot([1/np.sqrt(2),1/np.sqrt(2)*1j],hopi)*np.dot([1/np.sqrt(2),-1/np.sqrt(2)*1j],hopf)
#         print(np.dot([1/np.sqrt(2),1/np.sqrt(2)*1j],hopi),np.dot([1/np.sqrt(2),-1/np.sqrt(2)*1j],hopf))
#         C2 = 0
    elif (symm == 2): # B1g
        C1 = np.dot([1,0],hopi)*np.dot([1,0],hopf)
        C2 = -np.dot([0,1],hopi)*np.dot([0,1],hopf)
    elif (symm == 3): # B2g
        C1 = np.dot([1,0],hopi)*np.dot([0,1],hopf)
        C2 = np.dot([0,1],hopi)*np.dot([1,0],hopf)
    return C1, C2

# print(pol([1,1],[-1,1],1))

def hash_chiral(a,b,c,pts):
    # Hash a (S_a x S_b) * S_c term to a number
#     print("Hash: ", a,b,c, (a-1)*(pts**2)+(b-1)*pts+(c-1))
    if ((a>pts) or (b>pts) or (c>pts)): return -1
    if (len([a,b,c]) != len(set([a,b,c]))): return -1
    return (a-1)*(pts**2)+(b-1)*pts+(c-1)

def rhash_chiral(num,pts):
    # Return a number to a (S_a x S_b) * S_c term
    if (num >= pts**3): return -1,-1,-1
    a = int(num/(pts**2))+1
    num = num - (a-1)*(pts**2)
    b = int(num/pts)+1
    c = num - (b-1)*pts +1
    if (len([a,b,c]) != len(set([a,b,c]))): return -1,-1,-1
    return a,b,c

def hash_4term(a,b,c,d,pts):
    if ((a>pts) or (b>pts) or (c>pts) or (c>pts)): return -1
    return (a-1)*(pts**3)+(b-1)*(pts**2)+(c-1)*pts+(d-1)

def rhash_4term(num,pts):
    # Return a number to a (S_a x S_b) * S_c term
    if (num >= pts**4): return -1,-1,-1,-1
    a = int(num/(pts**3))+1
    num = num - (a-1)*(pts**3)
    b = int(num/(pts**2))+1
    num = num - (b-1)*(pts**2) +1
    c = int(num/pts)+1
    d = num - (c-1)*pts +1
    return a,b,c,d

def term_at(term_arr,ind,val):
    if (ind >= 0): term_arr[ind] += val
    return
                
def generate_perm(paths,pts=6,print_mode=1,symm=1,term=0):
    if (term == 2): term_arr = np.full((pts**4),0.0)
    elif (term == 1): term_arr = np.full((pts**3),0.0)
    else: term_arr = np.full((pts**2),0)
    for p in paths:
        print(p)
        hop1 = points_arr[p[1]-1]-points_arr[p[0]-1]
        hop2 = points_arr[p[2]-1]-points_arr[p[1]-1]
        hop3 = points_arr[p[3]-1]-points_arr[p[2]-1]
        hop4 = points_arr[p[4]-1]-points_arr[p[3]-1]
#         print(p,hop1,hop2,hop3,hop4)
        # Path1: v1->v2, v2->v3, v3->v4, v4->v1
#         if(not_parallel(hop1,hop4) and check_path_holes_4(p-1,pts,[1]) == 1):
        if(check_path_holes_4(p-1,pts,[1]) == 1):
            C1, C2 = pol(hop1,hop4,symm)
            if (term == 2):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+
                      ', T2a: -S_'+str(p[3])+","+str(p[2])+","+str(p[1])+","+str(p[0]))
                term_arr[hash_4term(p[3],p[2],p[1],p[0],pts)] += -(C1+C2)
            elif (term == 1):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2a:  S_'+str(p[2])+","+str(p[1])+","+str(p[0])
                      +' + S_'+str(p[3])+","+str(p[1])+","+str(p[0])
                  +' + S_'+str(p[3])+","+str(p[2])+","+str(p[0])+' - S_'+str(p[3])+","+str(p[2])+","+str(p[1]))
                term_at(term_arr,hash_chiral(p[2],p[1],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[1],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[1],pts),-(C1+C2))
            else: 
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2a:  S_'+str(p[3])+","+str(p[2])
                      +' + S_'+str(p[3])+","+str(p[1])+' - S_'+str(p[3])+","+str(p[0])
                      +' + S_'+str(p[2])+","+str(p[1])+' - S_'+str(p[2])+","+str(p[0])
                      +' - S_'+str(p[1])+","+str(p[0]))
                term_arr[hash_dot(p[3],p[2],pts)] += (C1+C2)
                term_arr[hash_dot(p[3],p[1],pts)] += (C1+C2)
                term_arr[hash_dot(p[3],p[0],pts)] += -(C1+C2)
                term_arr[hash_dot(p[2],p[1],pts)] += (C1+C2)
                term_arr[hash_dot(p[2],p[0],pts)] += -(C1+C2)
                term_arr[hash_dot(p[1],p[0],pts)] += -(C1+C2)
        # Path2: v1->v2, v2->v3, HOLON: v1->v4, DOUBLON: v3->v4
        if(check_path_holes_4(p-1,pts,[2]) == 1):
            C1, C2 = pol(hop1,hop3,symm)
            if (term == 2):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+
                      ', T2b: S_'+str(p[3])+","+str(p[2])+","+str(p[1])+","+str(p[0]))
                term_arr[hash_4term(p[3],p[2],p[1],p[0],pts)] += (C1+C2)
            elif (term == 1):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2b: -S_'+str(p[2])+","+str(p[1])+","+str(p[0])
                      +' + S_'+str(p[3])+","+str(p[1])+","+str(p[0])
                  +' + S_'+str(p[3])+","+str(p[2])+","+str(p[0])+' - S_'+str(p[3])+","+str(p[2])+","+str(p[1]))
                term_at(term_arr,hash_chiral(p[2],p[1],p[0],pts),-(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[1],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[1],pts),-(C1+C2))
            else: 
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2b:  -S_'+str(p[3])+","+str(p[2])
                      +' - S_'+str(p[3])+","+str(p[1])+' + S_'+str(p[3])+","+str(p[0])
                      +' + S_'+str(p[2])+","+str(p[1])+' - S_'+str(p[2])+","+str(p[0])
                      +' - S_'+str(p[1])+","+str(p[0]))
                term_arr[hash_dot(p[3],p[2],pts)] += -(C1+C2)
                term_arr[hash_dot(p[3],p[1],pts)] += -(C1+C2)
                term_arr[hash_dot(p[3],p[0],pts)] += (C1+C2)
                term_arr[hash_dot(p[2],p[1],pts)] += (C1+C2)
                term_arr[hash_dot(p[2],p[0],pts)] += -(C1+C2)
                term_arr[hash_dot(p[1],p[0],pts)] += -(C1+C2)
        # Path3: v1->v2, HOLON: v1->v4, DOUBLON: v2->v3, v3->v4
        if(check_path_holes_4(p-1,pts,[3]) == 1):
            C1, C2 = pol(hop1,hop3,symm)
            if (term == 2):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+
                      ', T2c: -S_'+str(p[3])+","+str(p[2])+","+str(p[1])+","+str(p[0]))
                term_arr[hash_4term(p[3],p[2],p[1],p[0],pts)] += (C1+C2)
            elif (term == 1):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2c: -S_'+str(p[2])+","+str(p[1])+","+str(p[0])
                      +' + S_'+str(p[3])+","+str(p[1])+","+str(p[0])
                  +' + S_'+str(p[3])+","+str(p[2])+","+str(p[0])+' - S_'+str(p[3])+","+str(p[2])+","+str(p[1]))
                term_at(term_arr,hash_chiral(p[2],p[1],p[0],pts),-(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[1],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[1],pts),-(C1+C2))
            else: 
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2c:  -S_'+str(p[3])+","+str(p[2])
                      +' - S_'+str(p[3])+","+str(p[1])+' + S_'+str(p[3])+","+str(p[0])
                      +' + S_'+str(p[2])+","+str(p[1])+' - S_'+str(p[2])+","+str(p[0])
                      +' - S_'+str(p[1])+","+str(p[0]))
                term_arr[hash_dot(p[3],p[2],pts)] += -(C1+C2)
                term_arr[hash_dot(p[3],p[1],pts)] += -(C1+C2)
                term_arr[hash_dot(p[3],p[0],pts)] += (C1+C2)
                term_arr[hash_dot(p[2],p[1],pts)] += (C1+C2)
                term_arr[hash_dot(p[2],p[0],pts)] += -(C1+C2)
                term_arr[hash_dot(p[1],p[0],pts)] += -(C1+C2)
        # Path4: v1->v2, HOLON: v1->v4, v4->v3, DOUBLON: v2->v3
        if(check_path_holes_4(p-1,pts,[4]) == 1):
            C1, C2 = pol(hop1,hop2,symm)
            if (term == 2):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+
                      ', -T2d:  S_'+str(p[3])+","+str(p[2])+","+str(p[1])+","+str(p[0]))
                term_arr[hash_4term(p[3],p[2],p[1],p[0],pts)] += -(C1+C2)
            elif (term == 1):
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2d: -S_'+str(p[2])+","+str(p[1])+","+str(p[0])
                      +' - S_'+str(p[3])+","+str(p[1])+","+str(p[0])
                  +' + S_'+str(p[3])+","+str(p[2])+","+str(p[0])+' - S_'+str(p[3])+","+str(p[2])+","+str(p[1]))
                term_at(term_arr,hash_chiral(p[2],p[1],p[0],pts),-(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[1],p[0],pts),-(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[0],pts),(C1+C2))
                term_at(term_arr,hash_chiral(p[3],p[2],p[1],pts),-(C1+C2))
            else: 
                print('C1,2: '+fmt_cmplx(C1)+","+fmt_cmplx(C2)+', T2d:  S_'+str(p[3])+","+str(p[2])
                      +' - S_'+str(p[3])+","+str(p[1])+' + S_'+str(p[3])+","+str(p[0])
                      +' - S_'+str(p[2])+","+str(p[1])+' + S_'+str(p[2])+","+str(p[0])
                      +' - S_'+str(p[1])+","+str(p[0]))
                term_arr[hash_dot(p[3],p[2],pts)] += (C1+C2)
                term_arr[hash_dot(p[3],p[1],pts)] += -(C1+C2)
                term_arr[hash_dot(p[3],p[0],pts)] += (C1+C2)
                term_arr[hash_dot(p[2],p[1],pts)] += -(C1+C2)
                term_arr[hash_dot(p[2],p[0],pts)] += (C1+C2)
                term_arr[hash_dot(p[1],p[0],pts)] += -(C1+C2)
#     term_arr[0] += 1j
#         print(term_arr)
    if (term == 2): print_4term(term_arr,pts,print_mode,symm)
    elif (term == 1): print_chiral(term_arr,pts,print_mode)
    else: print_dot(term_arr,pts,print_mode,symm,1)
    return

def fmt_str(pt,print_mode=2):
    ptstr = ""
    if (print_mode == 2):
        x = points_arr[pt][0]
        if (x > 1): ptstr += ("+"+str(x)+"x")
        elif (x < -1): ptstr += (str(x)+"x")
        elif (x == 1): ptstr += ("+x")
        elif (x == -1): ptstr += ("-x")
        y = points_arr[pt][1]
        if (y > 1): ptstr += ("+"+str(y)+"y")
        elif (y < -1): ptstr += (str(y)+"y")
        elif (y == 1): ptstr += ("+y")
        elif (y == -1): ptstr += ("-y")
    elif (print_mode == 3):
        x = points_arr[pt][0]
        y = points_arr[pt][1]
        ptstr += (str(x) + "," + str(y))
    return ptstr

def fmt_cmplx(val,print_mode=1):
    if isinstance(val, complex):
        if (isinstance(val.real, int)): real = str(int(val.real))
        else: 
            if (val.real.is_integer()): real = str(int(val.real))
            else: real =  "{:.0f}".format(val.real)
        if (isinstance(val.imag, int)): imag = str(int(val.imag))
        else: 
            if (val.imag.is_integer()): imag = str(int(val.imag))
            else: imag =  "{:.0f}".format(val.imag)
#         print("complex",val.real,val.imag,real,imag)
        if (print_mode == 1 or print_mode == 2):
            if (imag[0] != "-"): return "(" + real + "+" + imag + "i)"
            else: return "(" + real + imag + "i)"
        else: return "cmplx(" + real + "," + imag + ")"
    elif (isinstance(val, int)):
        return str(int(val))
    else: 
        if (abs(val % 1) < 1e-5): 
            if (print_mode == 4):
                if (val > 0): return "+"+str(int(val))
                else: return str(int(val))
            return str(int(val))
        else: return "{:.3f}".format(val)
    return val

def permute_path(path):
    # Generate cyclic permutation and reverse pathway for a given path
    start_point = path[0]
    all_path = np.asarray([path])
#     all_path = np.vstack((all_path,path,np.flip(path)))
    path = path[:-1]
    for i in range(3):
        path = np.roll(path,1)
#         print(np.append(path,path[0]))
        if (path[0] == start_point or path[1] == start_point): 
            all_path = np.vstack((all_path,np.append(path,path[0])))
    path = np.roll(path,1)
    path = np.flip(path)
    if (path[0] == start_point or path[1] == start_point): 
        all_path = np.vstack((all_path,np.append(path,path[0])))
    for i in range(0,3):
        path = np.roll(path,1)
        if (path[0] == start_point or path[1] == start_point): 
            all_path = np.vstack((all_path,np.append(path,path[0])))
    return np.unique(all_path,axis=0)

def valid_path(path,pts,order):
    check = np.full((pts),1)
#     print("new path")
    for hop in range(len(order)):
        check[path[order[hop]-1]] -= 1
        check[path[order[hop]]] += 1
#         print(check, np.count_nonzero(check == 2))
        if (np.array_equal(check,np.full((pts),1))): return False
    return True

def check_path_holes_4(path,pts,pp=[]):
    prefac = 0
    for p in pp:
        if (p == 1):
            if (valid_path(path,pts,[1,2,3])): prefac += 1
        elif (p == 2):
            if (valid_path(path,pts,[1,2,4])): prefac += 1
        elif (p == 3):
            if (valid_path(path,pts,[1,4,2])): prefac += 1
        elif (p == 4):
            if (valid_path(path,pts,[1,4,3])): prefac += 1
    return prefac

# check_path_holes_4(np.asarray([1,2,4,2,1])-1,4,[1,2,3,4])
# permute_path([1,2,3,4,1])
randm = np.random.rand(5,5)

In [ ]:
def meet_crit(path,check,hop_num):
    if (path[len(path)-1] != path[0]): return False
    if (all_parallel(path)): return False #### I THINK THIS IS NECESSARY for counting in A2g
    if (not check(path)): return False
    if (not np.array_equal(hop_num,check_hop(path))): return False # Check hop_num is all zero
#     print(not np.array_equal(hop_num,check_hop(path)),hop_num,check_hop(path),path+1)
    return True

def explore(hops,curpt,path,gridnum,patharr,check=default_check,hop_num=[2,2,0,0],gen_perm=True):
    if (hops == 0):
        path = np.append(path,[curpt])
        path = path.astype(int)
#         print(path+1,hop_num,check_hop(path))
        if(meet_crit(path,check,hop_num)):
#             print(path+1,patharr,patharr.shape)
            if (not gen_perm):
                if (len(patharr) == 0): patharr = np.append(patharr,path+1).astype(int)
                else: patharr = np.vstack((patharr,path+1)).astype(int)
            else:
                if (len(patharr) == 0): patharr = permute_path(path+1).astype(int)
                else: patharr = np.vstack((patharr,permute_path(path+1))).astype(int)
    else:
        hops = hops - 1
        # Kick out paths that are already ineligibe
        if (len(path) >= 2):
            hn = check_hop(path.astype(int))
            for n,h in enumerate(hn):
                if (h > hop_num[n]): return patharr
            if (np.sum(hn) != (len(path)-1)): return patharr # Path out of range
#             print(path.astype(int))
        ######
        for point in range(gridnum):
            if (curpt != point): 
                patharr = explore(hops, point, np.append(path,[curpt]),gridnum,patharr,check,hop_num,gen_perm)
    return patharr



In [ ]:
# Check: One Square, only NN hops
## Generate permutation, if not include 1 will generate extra term

# 3---------4
# |         |
# |         |  y
# |         |  |
# 1---------2  --> x

points_arr = np.asarray([[0,0],[1,0],[0,1],[1,1]])
# points_arr = np.asarray([[0,0],[1,0],[0,-1],[1,-1]])

paths_arr = np.asarray([])
paths_arr = explore(4,0,np.array([]),points_arr.shape[0],paths_arr,hop_num=[4,0,0,0],gen_perm=False)
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
# paths_arr = np.asarray([[1,2,4,2,1]])
generate_perm(paths_arr,4,2,1,1)


In [ ]:
# CASE 3: One Square

# 3---------1---------2  --> x
def check_span(path):
#     if (len(path) == len(set(path))+1): return False
    return True #(path[1] in [3])

points_arr = np.asarray([[0,0],[1,0],[-1,0]])
paths_arr = np.asarray([])
paths_arr = explore(4,0,np.array([]),points_arr.shape[0],paths_arr,check=check_span,hop_num=[4,0,0,0],gen_perm=False)
# print("Point 1: ",len(paths_arr))
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
generate_perm(paths_arr,3,3,1,1)


In [ ]:
# CASE 2

# 6---------3---------4
# |         |         |
# |         |         | 
# |         |         | 
# 5---------1---------2
# |         |         |
# |         |         |  y
# |         |         |  |
# 7---------8---------9  --> x


def check_span(path):
#     if (len(path) == len(set(path))+1): return False
    return True
    all_pt = np.asarray([1,2,3,4])-1
    if (set(path).issubset(all_pt)): return True
    return False #(path[1] in [3])

points_arr = np.asarray([[0,0],[1,0],[0,1],[1,1],[-1,0],[-1,1],[-1,-1],[0,-1],[1,-1]])
paths_arr = np.asarray([])
paths_arr = explore(2,0,np.array([]),points_arr.shape[0],paths_arr,check=check_span,hop_num=[2,0,0,0],gen_perm=False)
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
print(paths_arr)
# paths_arr = np.asarray([[1,2,3,1],[2,1,3,2]])
# generate_perm(paths_arr,9,3,1,1)

# EXAMPLE: 6 Hops #

In [ ]:
def check_path_holes_6(path,pts,pp=[]):
    prefac = 0
    for p in pp:
        if (p == 1):
            if (valid_path(path,pts,[1,2,3,4,5])): prefac += 1
        elif (p == 2):
            if (valid_path(path,pts,[1,2,3,4,6])): prefac += 1
        elif (p == 3):
            if (valid_path(path,pts,[1,2,3,6,4])): prefac += 1
        elif (p == 4):
            if (valid_path(path,pts,[1,2,3,6,5])): prefac += 1
        elif (p == 5): 
            if (valid_path(path,pts,[1,2,6,3,4])): prefac += 1
        elif (p == 6):
            if (valid_path(path,pts,[1,2,6,3,5])): prefac += 1
        elif (p == 7):
            if (valid_path(path,pts,[1,2,6,5,3])): prefac += 1
        elif (p == 8):
            if (valid_path(path,pts,[1,2,6,5,4])): prefac += 1
        elif (p == 9):
            if (valid_path(path,pts,[1,6,2,3,4])): prefac += 1
        elif (p == 10):
            if (valid_path(path,pts,[1,6,2,3,5])): prefac += 1
        elif (p == 11):
            if (valid_path(path,pts,[1,6,2,5,3])): prefac += 1
        elif (p == 12):
            if (valid_path(path,pts,[1,6,2,5,4])): prefac += 1
        elif (p == 13):
            if (valid_path(path,pts,[1,6,5,2,3])): prefac += 1
        elif (p == 14):
            if (valid_path(path,pts,[1,6,5,3,4])): prefac += 1
        elif (p == 15):
            if (valid_path(path,pts,[1,6,5,4,2])): prefac += 1
        elif (p == 16):
            if (valid_path(path,pts,[1,6,5,4,3])): prefac += 1
    return prefac

In [ ]:
def generate_perm_6(paths,pts=6,print_mode=1,symm=1,term=0):
    if (term == 2): term_arr = np.full((pts**4),0.0)
    elif (term == 1): term_arr = np.full((pts**3),0.0)
    else: term_arr = np.full((pts**2),0)
    for p in paths:
        print(p)
        hop1 = points_arr[p[1]-1]-points_arr[p[0]-1]
        hop2 = points_arr[p[2]-1]-points_arr[p[1]-1]
        hop3 = points_arr[p[3]-1]-points_arr[p[2]-1]
        hop4 = points_arr[p[4]-1]-points_arr[p[3]-1]
        hop5 = points_arr[p[5]-1]-points_arr[p[4]-1]
        hop6 = points_arr[p[6]-1]-points_arr[p[5]-1]
#         print(p,hop1,hop2,hop3,hop4)
        if(check_path_holes_6(p-1,pts,[1])!=0):
            C1, C2 = pol(hop1,hop6,symm)
            prefac = check_path_holes_6(p-1,pts,[1])
            print("Hop 1,6. Prefac: ",prefac,", C1:",C1,", C2:",C2)
            if (term == 1):
                term_arr = fill_term(term_arr,(C1+C2)*prefac,[1,-1,-1,-1,-1,-1],p,pts,term)
        # Path2: v1->v2, v2->v3, HOLON: v1->v4, DOUBLON: v3->v4
        if(check_path_holes_6(p-1,pts,[2,3,5,9])!=0):
            C1, C2 = pol(hop1,hop5,symm)
            prefac = check_path_holes_6(p-1,pts,[2,3,5,9])
            print("Hop 1,5. Prefac: ",prefac,", C1:",C1,", C2:",C2)
            if (term == 1):
                term_arr = fill_term(term_arr,(C1+C2)*prefac,[-1,1,1,1,1,-1],p,pts,term)
        # Path3: v1->v2, HOLON: v1->v4, DOUBLON: v2->v3, v3->v4
        if(check_path_holes_6(p-1,pts,[4,6,7,10,11,13])!=0):
            C1, C2 = pol(hop1,hop4,symm)
            prefac = check_path_holes_6(p-1,pts,[4,6,7,10,11,13])
            print("Hop 1,4. Prefac: ",prefac,", C1:",C1,", C2:",C2)
            if (term == 1):
                term_arr = fill_term(term_arr,(C1+C2)*prefac,[1,-1,-1,-1,1,1],p,pts,term)
        # Path4: v1->v2, HOLON: v1->v4, v4->v3, DOUBLON: v2->v3
        if(check_path_holes_6(p-1,pts,[8,12,14,15])!=0):
            C1, C2 = pol(hop1,hop3,symm)
            prefac = check_path_holes_6(p-1,pts,[8,12,14,15])
            print("Hop 1,3. Prefac: ",prefac,", C1:",C1,", C2:",C2)
            if (term == 1):
                term_arr = fill_term(term_arr,(C1+C2)*prefac,[-1,1,1,-1,-1,-1],p,pts,term)
        if(check_path_holes_6(p-1,pts,[16])!=0):
            C1, C2 = pol(hop1,hop2,symm)
            prefac = check_path_holes_6(p-1,pts,[16])
            print("Hop1,2. Prefac: ",prefac,", C1:",C1,", C2:",C2)
            if (term == 1):
                term_arr = fill_term(term_arr,(C1+C2)*prefac,[1,-1,1,1,1,1],p,pts,term)
#     term_arr[0] += 1j
#         print(term_arr)
    if (term == 2): print_4term(term_arr,pts,print_mode,symm)
    elif (term == 1): print_chiral(term_arr,pts,print_mode)
    else: print_dot(term_arr,pts,print_mode,symm,1)
    return

In [ ]:
# Check: One Square, only NN hops

# 3---------4
# |         |
# |         |  y
# |         |  |
# 1---------2  --> x

points_arr = np.asarray([[0,0],[1,0],[0,1],[1,1]])
# points_arr = np.asarray([[0,0],[1,0],[0,-1],[1,-1]])

paths_arr = np.asarray([])
paths_arr = explore(6,0,np.array([]),points_arr.shape[0],paths_arr,hop_num=[6,0,0,0],gen_perm=False)
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
# paths_arr = np.asarray([[1,2,4,2,1]])
# print(paths_arr)

generate_perm_6(paths_arr,4,3,1,1)


In [ ]:
def check_span(path):
    second_pt = np.asarray([3])-1
#     return True
    has24 = False
    has56 = False
    if (2 in path+1 or 4 in path+1): has24 = True
    if (5 in path+1 or 6 in path+1): has56 = True
#     if (has24 and has56 and (path[1] in second_pt)): print(path[1])
    return (has24 and has56)
#     return False
points_arr = np.asarray([[0,0],[1,0],[0,1],[1,1],[-1,0],[-1,1]])
paths_arr = np.asarray([])
     
# for hop1 in range(0,6):
#     print(hop1)
paths_arr = explore(6,0,np.array([]),points_arr.shape[0],paths_arr,check=check_span,hop_num=[6,0,0,0],gen_perm=False)
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
# print(paths_arr)
# paths_arr = np.asarray([[1,4,1,5,1],[1,5,1,4,1]])
generate_perm_6(paths_arr,6,3,1,1)
# CASE 1: staring point in the middle
# 6---------3---------4
# |         |         |
# |         |         |  y
# |         |         |  |
# 5---------1---------2  --> x

In [ ]:
# CASE 2.1: staring point in the corner, 2x

# 3---------4---------6
# |         |         |
# |         |         |  y
# |         |         |  |
# 1---------2---------5  --> x


def check_span(path):
#     second_pt = np.asarray([1,2,3,4,5,6])-1
    has56 = False
    if (5 in path+1 or 6 in path+1): has56 = True
    return (has56)

points_arr = np.asarray([[0,0],[1,0],[0,1],[1,1],[2,0],[2,1]])
paths_arr = np.asarray([])
paths_arr = explore(6,0,np.array([]),points_arr.shape[0],paths_arr,check=check_span,hop_num=[6,0,0,0],gen_perm=False)
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
print(paths_arr)
# paths_arr = np.asarray([[1,2,6,2,1],[1,2,6,4,1]])
generate_perm_6(paths_arr,6,3,1,1)

In [ ]:
# CASE 2: staring point in the corner, 2x

# 7---------8---------9
# |         |         |
# |         |         | 
# |         |         | 
# 4---------5---------6
# |         |         |
# |         |         |  y
# |         |         |  |
# 1---------2---------3  --> x


def check_span(path):
#     second_pt = np.asarray([1,2,3,4,5,6])-1
#     return True
    has78 = False
    has36 = False
    if (3 in path+1 or 6 in path+1): has36 = True
    if (7 in path+1 or 8 in path+1): has78 = True
    return (has36 or has78)

points_arr = np.asarray([[0,0],[1,0],[2,0],[0,1],[1,1],[2,1],[0,2],[1,2],[2,2]])
paths_arr = np.asarray([])
paths_arr = explore(6,0,np.array([]),points_arr.shape[0],paths_arr,check=check_span,hop_num=[6,0,0,0],gen_perm=False)
print(len(paths_arr),len(np.unique(paths_arr,axis=0)))
paths_arr = np.unique(paths_arr,axis=0)
generate_perm_6(paths_arr,9,3,1,1)

In [ ]:
# CASE FINAL: All Hopping Pathways

# 18-------17--------11--------16--------15
# |         |         |         |         |
# |         |         |         |         | 
# |         |         |         |         |
# 19--------7---------3---------6--------14 
# |         |         |         |         |
# |         |         |         |         | 
# |         |         |         |         |  
# 12--------4---------1---------2--------10
# |         |         |         |         |
# |         |         |         |         | 
# |         |         |         |         | 
# 20--------8---------5---------9--------25 
# |         |         |         |         |
# |         |         |         |         |  y
# |         |         |         |         |  |
# 21--------22-------13--------23--------24  --> x


def check_span(path):
    return True
    all_pt = np.asarray([1,3,4,7,11])-1
    if (set(path).issubset(all_pt)): return False
    all_pt = np.asarray([1,2,10,3,6,14,11,16,15,4,7])-1
    if (set(path).issubset(all_pt)): return True
    return False

points_arr = np.asarray([[0,0],[1,0],[0,1],[-1,0],[0,-1],[1,1],[-1,1],[-1,-1],[1,-1],
                        [2,0],[0,2],[-2,0],[0,-2],[2,1],[2,2],[1,2],[-1,2],[-2,2],
                        [-2,1],[-2,-1],[-2,-2],[-1,-2],[1,-2],[2,-2],[2,-1]])

# points_arr = np.asarray([[0,0],[1,0],[1,1],[0,1],[-1,1],[-1,0],[-1,-1],[0,-1],[1,-1],[2,-1],
#                         [2,0],[2,1],[2,2],[1,2],[0,2],[-1,2],[-2,2],[-2,1],[-2,0],[-2,-1],
#                          [-2,-2],[-1,-2],[0,-2],[1,-2],[2,-2]])
paths_arr_long = np.asarray([])
# print(len(paths_arr_long))
paths_arr_long = explore(6,0,np.array([]),points_arr.shape[0],paths_arr_long,
                         check=check_span,hop_num=[6,0,0,0],gen_perm=False)
